# Libraries

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install CatBoost
!pip install country_converter
!pip install lime -q

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
import time
import plotly.express as px
import plotly.graph_objects as go
import shap
from matplotlib import cm
from tqdm import tqdm
!pip install biopython
from Bio.Seq import Seq
from lime.lime_tabular import LimeTabularExplainer

# Sklearn Module
from sklearn.model_selection import KFold,GroupKFold,GridSearchCV,StratifiedKFold
from sklearn.metrics import confusion_matrix,balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer,RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.model_selection import learning_curve,ShuffleSplit
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE, SMOTENC, SMOTEN

# Machine Learning Models
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import os,warnings;warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Reading Files

In [ ]:
azm_sr_df = pd.read_csv('/content/drive/MyDrive/Dataset/azm_sr_gwas_filtered_unitigs.Rtab', sep="\t")
cfx_sr_df = pd.read_csv('/content/drive/MyDrive/Dataset/cfx_sr_gwas_filtered_unitigs.Rtab', sep="\t")
cip_sr_df = pd.read_csv('/content/drive/MyDrive/Dataset/cip_sr_gwas_filtered_unitigs.Rtab', sep="\t")
metadata_df = pd.read_csv('/content/drive/MyDrive/Dataset/metadata.csv')

# Basic Stats

# **1. Introduction**

## 1.1 Background

RESISTANCE OF BACTERIA

  We will be focussing on a species called Neisseria gonorrhoeae, the bacteria which cause gonorrhoea.
Gonorrhoea is the second most common sexually transmitted infection (STI) in Europe, after chlamydia.
Rates of gonorrhoea infection are on the rise, with a 26% increase reported from 2017-2018 in the UK.

  Many people who are infected (especially women) experience no symptoms, helping the disease to spread.
If the infection is left untreated, it can lead to infertility in women, and can occasionally spread to
other parts of the body such as your joints, heart valves, brain or spinal cord.
Resistance of these bacteria to antibiotics is rising over time, making infections hard to treat.

* In the past, patients were treated with an antibiotic called ciprofloxaxcin.
* Doctors had to stop using this antibiotic because resistance to the drug became too common, causing treatments of infections to fail.
* Until very recently, the recommended treatment was two drugs - ceftriaxone and azithromycin.
* Azithromycin was removed from recommendations because of concern over rising resistance to the antibiotic.
* In February 2018, the first ever reported case of resistance to treatment with ceftriaxone and azithromycin, as well as resistance to the last-resort treatment spectinomycin, was reported.
* Currently in the UK, patients are only treated with ceftriaxone.

*ANTIBIOTICS*

Three antibiotics and associated unitigs are used to make a model in this problem.

*Azithromycin* :
* Azithromycin is an antibiotic used to treat various types of infections of the respiratory tract, ear, skin and eye in adults and children. It is also effective in typhoid fever and some sexually transmitted diseases like gonorrhea.

*Ciprofloxacin* :
* Ciprofloxacin is an antibiotic, used in the treatment of bacterial infections. It is also used in treating infections of the urinary tract, nose, throat, skin and soft tissues and lungs (pneumonia). It prevents the bacterial cells from dividing and repairing, thereby killing them.

*Cefixime* :
* Cefixime is an antibiotic medicine used to treat a variety of bacterial infections. It is effective in infections of the respiratory tract (eg. pneumonia), urinary tract, ear, nasal sinus, throat, and some sexually transmitted diseases.

**UNITIGS**

In our dataset, we will come across features data that will convey the presence or absence of a particular nucleotide sequence in the Bacteria's DNA

For this analysis, we're using unitigs, stretches of DNA (in string format) shared by a subset of the strains in our study.
* Unitigs are an efficient but flexible way of representing DNA variation in bacteria.
* The full dataset consists of 584,362 unitigs, which takes a long time to train models on, so for this exercise
* We will be using a set that has been filtered for unitigs associated with resistance.


## 1.2. Target Variables

*Neisseria gonorrhoeae* is either resistant (target=1) to a particular treatment or not resistant (target=0) for a particular.

AVAILABLE UNITIG DATA:

* We will be choosing one of the following antibiotic cases below:

  1.   Bacteria resistance to *Azithromycin; azm_sr*
  2.   Bacteria resistance to *Ciprofloxacin; cip_sr*
  3.   Bacteria resistance to *Cefixime; cfx_sr*

UNAVAILABLE UNITIG DATA:

* We don't currently have unitig data for the following antibiotics, so we can't check these cases:

  1.   Bacteria resistance to *Ceftriaxone; cro_sr*
  2.   Bacteria resistance to *Tetracycline; tet_sr*
  3.   Bacteria reistance to *Penicillin; pen_sr*

# 2. Dataset

## 2.1. Creating Dataset

**GET_UNITIGS CLASS**

  *   We need to do some data wrangling & combine two data files together; `.get_case`
  *   In this problem, a case type format is used (grouped feature matrix & target vector)

The feature matrix (stored within .X) will contain:

  * All collected samples from across various databses, corresponding to the bacteria's DNA (which was cut into segments of 31-mers)

  * When a particular DNA segment (unitig) is present in the bacterial sample DNA, it's value is set to 1 & 0 if it's not present

**MERGING RTAB & METADATA**

  * Filtered Unitig data is located in .Rtab files, corresponding to the same **`sample_id`** as that of the metadata

  * We need read the .Rtab files & merge the two dataframes based on the **`sample_id`** index; using .get_case()



In [ ]:
''' Align Metadata Target values Unitig File & Compile Feature Matrix '''

class get_unitigs:

    def __init__(self,verbose=True):
        self.df = pd.read_csv('/content/drive/MyDrive/Dataset/metadata.csv', index_col=0) # metadata
        self.meta_names = self.df.columns
        self.target_name = None
        self.verbose = verbose

    # Get Unitig Feature matrix & Target Vector
    def get_case(self,phenotype=None):

        self.target_name = phenotype
        _metadata = self.df
        if(self.verbose):
            print(f'Target Antibiotic: {self.target_name}')
            print(f'Metadata df: {_metadata.shape}')

        # remove those that don't contain target values
        _metadata = _metadata.dropna(subset=[phenotype])
        self.metadata = _metadata.copy()

        if(self.verbose):
            print(f'Metadata df after na() removal {_metadata.shape}')
        _metadata = _metadata[phenotype] # choose target variable

        prefix = '/content/drive/MyDrive/Dataset/'
        suffix = '_gwas_filtered_unitigs.Rtab'

        if(self.verbose):
            print('\nCombining Metadata & Unitigs')

        # unitig feature matrix for phenotype
        tdf = pd.read_csv(prefix + phenotype + suffix, sep=" ",
                          index_col=0, low_memory=False)
        # align column data w/ metadata df (pattern_id = sample_idd)
        tdf = tdf.T
        # keep only common rows, ie. that have resistence measure]
        tdf = tdf[tdf.index.isin(_metadata.index)]

        train = tdf
        target = _metadata[_metadata.index.isin(tdf.index)]

        self.X = pd.concat([train,target],axis=1)
        if(self.verbose):
            print(f'Unitig Matrix (+target): {self.X.shape}')

In [ ]:
metadata_df['Year'] = metadata_df['Year'].fillna(0).astype(np.int64)

## 2.2 Case Preview - Ciprofloxacin

  We'll be looking at bacterial resistance for three different cases, let's check one case first & load the other when required..
---

**METADATA CONTENT**

* Metadata is stored in .df, which contains our target variable (one of the _sr columns) for a specific Sample_ID

* As we can see, we don't have all the data, some data is missing in our metadata, we'll only be dropping Nan cases for the target variable (_sr) we're concerned with.

**FEATURE MATRIX**

* The feature matrix is created from one of the .Rtab files, depending on which case we are testing.

* Each column in the feature matrix is called a unitig (pattern_id); is treated as a feature for the specific Sample_ID, which is our index.

In [ ]:
# Load meta data & look at the first 5 samples
case_cip = get_unitigs()

display(case_cip.df.T.iloc[:,:7])

In [ ]:
case_cip.get_case(phenotype='cip_sr')
case_cip.X.iloc[:,:2].head()

**TARGET VARIABLE**

* The target vector, is also stored in .X, and represents the resistance property of the sample
* The Sample_ID is either 1.0 (resistant) or 0.0 (non-resistant) to a particular antibiotic in question.

In [ ]:
target = case_cip.X[case_cip.target_name]
print(target[:5])

## 2.3. Target Variable Distribution

**TARGET DISTRIBUTIONS & UNITIG COUNT**

* It's useful to look at the class distributions for all three cases we'll be making models for.
* As we can see below, the unitig distributions are quite different as well, so our feature matrix will vary significantly for each case.

In [ ]:
''' Ciprofloxacin '''

case = get_unitigs()
case.get_case('cip_sr')
print(case.X[case.target_name].value_counts())

In [ ]:
''' Azithromycin '''

case = get_unitigs()
case.get_case('azm_sr')
print(case.X[case.target_name].value_counts())

In [ ]:
''' Cefixime '''

case = get_unitigs()
case.get_case('cfx_sr')
print(case.X[case.target_name].value_counts())

# 3.Feature Matrix Modification

## 3.1.Creating Case Data

* In Section 2.3, we saw that for some antibiotics, there are very few resistant samples present , mainly Cefixime
* This results in a very tricky situation if we want to use cross validation, we may not even have enough samples for the model to learn anything meaningful
* Creating models Azithromycin & Ciprofloxacin should not have this issue, as we have sufficient number of samples, even if an imbalance is present

We can try two approaches:

* Downsample the dominant class (not resistant) `.split_case`
* Utilise SMOTE based upsampling strategy `smote`

In [ ]:
''' Feature Matrix Upsampling Modification (+Target) '''
# Model based approach to upsample minor class in target variable

class mod_unitigs():

    def __init__(self,unitigs):
        self.X = unitigs.X # input data class
        self.target_name = unitigs.target_name
        self.verbose = True

    ''' Downsampling Class 0 using .sample & recompile '''
    # If there's too much of the dominant class, just downsample

    def split_case(self,frac_id=0.5):

        X = self.train
        y = pd.Series(self.target,name=self.target_name)
        XX = pd.concat([X,y],axis=1)

        lst_temp = dict(tuple(XX.groupby(self.target_name))) # divide classes
        ratio = lst_temp[0].shape[0]/lst_temp[1].shape[0] # get class ratio

        # Sample approach for downsizing majority class
        X_red = lst_temp[0].sample(frac=frac_id)
        X_all = pd.concat([X_red,lst_temp[1]],axis=0)

        if(self.verbose):
            print(f'Class 0 : {lst_temp[0].shape}')
            print(f'Class 1 : {lst_temp[1].shape}')
            print(f'Class Ratio: {round(ratio,4)}')
            print(f'Reduced Training Matrix: {X_all.shape}')

        # Redefine .train, .target
        self.target = X_all[self.target_name].copy()
        X_all.drop(self.target_name, inplace=True, axis=1)
        self.train = X_all

    ''' SMOTE UPSAMPLING '''
    # For unbalanced problems, synthetically/model new data

    def smote(self,smote_id = 'smotenc',
                   smote_strat=0.5,
                   k_neighbours=5):

        self.smote_id = smote_id
        self.smote_strat = smote_strat
        self.smote_nbr = k_neighbours

        y = self.X[self.target_name].copy()
        X = self.X.drop([self.target_name],axis=1).copy()

        # smote for contin, smotenc for category
        if(self.smote_id == 'smote'):
            model_id = SMOTE(sampling_strategy=self.smote_strat,
                             k_neighbors=self.smote_nbr)
        elif(self.smote_id == 'smotenc'):
            model_id = SMOTENC(sampling_strategy=self.smote_strat,
                               k_neighbors=self.smote_nbr,
                               categorical_features=[0,1])

        X_mod, y_mod = model_id.fit_resample(X,y)
        self.X = pd.concat([X_mod,y_mod],axis=1)

        if(self.verbose):
            print(f'\nSMOTE Upsampling: {self.X.shape}')
            print(f'Target Value Counts: \n{pd.Series(y_mod).value_counts()}')
        self.X = pd.concat([X_mod,y_mod],axis=1)

# 4.Explotary Data Analysis

## 4.1. Parallel Categories

**EXPLORING THE METADATA**

* Aside from the unitig data, the Metadata Dataset contains some interesting info about each Sample_ID as well.
* Having sorted by the X_mic column, *log2_X_mic* has the same ordering as X_mic & antibiotic column name, so it was not included to reduce clutter in the figures.
* Beta.lactamase relation to antibiotic resistance doesn't seem to exhibit any particular patterns, other than Penicillin, for which the R (resistant) type almost exclsively indicates that the antibiotic will not be effective.
* Most bacteria *Sample_ID* are also shown to be of S (sensitive) type.
* If we highlight the uttermost right column, _sr, we can also note that resistance of these bacteria to antibiotics tends to be rising over time.
* These figures clearly indicate that *cefixime* by far is the most effective treatment out of the tree antibiotics.
* *Ciprofloxacin* on the otherhand has not been very effective treatment against the bacteria.

In [ ]:
lst_azm = ['Year','Country','Continent','Beta.lactamase','azm_mic','Azithromycin','azm_sr']
lst_cip = ['Year','Country','Continent','Beta.lactamase','cip_mic','Ciprofloxacin','cip_sr']
lst_cfx = ['Year','Country','Continent','Beta.lactamase','cfx_mic','Cefixime','cfx_sr']
lst_antibio = [lst_azm,lst_cip,lst_cfx]

# Plot Parallel Categories Plot
def plot_pp(lst,colour='ghostwhite'):
    tdf = get_unitigs().df[lst]
    tdf.dropna(inplace=True)
    tdf.sort_values(by=lst[-3],inplace=True,ascending=False)
    tdf['Year'] = tdf['Year'].astype(str)
    fig = px.parallel_categories(tdf)
    fig.update_traces(patch={"line": {"color":colour,'shape':'hspline'}})
    fig.update_layout(title=f'Bacteria Resistance to {lst[-2]}')
    fig.update_layout(margin=dict(t=60,b=10),height=400)
    fig.show()

def plot_geomean(lst):

    global country_map

    tdf = get_unitigs().df[lst]
    tdf.dropna(inplace=True)
    tdf.sort_values(by=lst[-3],inplace=True,ascending=False)
    # tdf['Year'] = tdf['Year'].astype(str) # Exclude 'Year' from mean calculation

    tdf['Country'] = tdf['Country'].map(country_map)
    # Select only numeric columns for mean calculation
    numeric_cols = tdf.select_dtypes(include=np.number).columns.tolist()
    tdf2 = tdf.groupby(['Country'])[numeric_cols].mean()

    fig = go.Figure(data=go.Choropleth(
        locations = tdf2.index,
        z = tdf2[lst[-1]],
        colorscale = 'magenta',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='black',
        marker_line_width=0.5,
        colorbar_title = f'{lst[-1]}'))

    fig.update_layout(title=f'Bacteria Resistance to {lst[-2]}',
                      geo=dict(showframe=False,showcoastlines=False,
                               projection_type='equirectangular'))
    fig.update_layout(margin=dict(t=60,b=10),height=400)
    fig.show()

In [ ]:
for i in lst_antibio:
    plot_pp(i,'mistyrose')

## 4.2. Country Based Bacteria Resistance

**LEAST EFFECTIVE TREATMENT LOCATIONS**

The result sample pool may not be very balanced to make specific conclusions, but resistance to specific anibiotics has some geographic variation.

* For Azithromycin, samples from the US, Sweden & China are shown to be only countries with unsuccesful treatment cases.
* Globally, Ciprofloxacin has become an innefective treatment. Chile, Finland, Vietnam, China are amonst the least effective locations.
* Aside from France, Cefixime has been the most effective treatment globally.

In [ ]:
import country_converter as coco

lst_azm = ['Year','Country','Continent','Beta.lactamase','azm_mic','Azithromycin','azm_sr']
lst_cip = ['Year','Country','Continent','Beta.lactamase','cip_mic','Ciprofloxacin','cip_sr']
lst_cfx = ['Year','Country','Continent','Beta.lactamase','cfx_mic','Cefixime','cfx_sr']
lst_antibio = [lst_azm,lst_cip,lst_cfx]

#tdf['Country'] = coco.convert(names=tdf['Country'].astype(str), to='ISO3', not_found=None)

def plot_geomean(lst):

    #global country_map

    tdf = get_unitigs().df[lst]
    tdf.dropna(inplace=True)
    tdf.sort_values(by=lst[-3],inplace=True,ascending=False)
    tdf['Year'] = tdf['Year'].astype(str)

    tdf['Country'] = coco.convert(names=tdf['Country'].replace(
        {'Brasil':'Brazil',
         'Scotland':'United Kingdom',
         'Caribbean':'Caribbean Netherlands'}).astype(str), to='ISO3', not_found=None)

    tdf2 = tdf.groupby(['Country']).mean(numeric_only=True)

    fig = go.Figure(data=go.Choropleth(
        locations = tdf2.index,
        z = tdf2[lst[-1]],
        colorscale = 'magenta',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='black',
        marker_line_width=0.5,
        colorbar_title = f'{lst[-1]}'))

    fig.update_layout(title=f'Bacteria Resistance to {lst[-2]}',
                      geo=dict(showframe=False,showcoastlines=False,
                               projection_type='equirectangular'))
    fig.update_layout(margin=dict(t=60,b=10),height=400)
    fig.show()

In [ ]:
# Plot Choropleth Map
for i in lst_antibio:
    plot_geomean(i)

## 4.3. MIC Values of All Samples

* *MIC* : a measure of the *concentration of antibiotic* bacteria can tolerate before it impairs their growth.
* We can definitely note a correlation to _mic features for antibiotic resistance in this graph alone:
  * Only small quantities of Cefixime are required to impair bacterial growth.
  * Compared to Azithromycin & Ciprofloxacin, which concentrate higher values of _mic as well
  * Hinting that it's less effective or that the bacteria is tending to become more resistant to the antibiotic & larger quantities are required to affect its function.

In [ ]:
def get_mic():

    lst_cases = ['azm_mic','cip_mic','cfx_mic']
    rtabs = ['azm_sr','cip_sr','cfx_sr']
    lst_temp = []

    ii=-1
    for case in lst_cases:

        ii+=1
        case_id = get_unitigs(verbose=False)
        case_id.get_case(rtabs[ii])

        X_all = pd.concat([case_id.X,case_id.metadata],axis=1)

        new_df = X_all[case].value_counts().rename_axis(case).reset_index(name='counts')
        new_df = new_df.rename(columns={new_df.columns[0]: 'mic'})
        new_df['case'] = case
        lst_temp.append(new_df)

    X_counts = pd.concat([lst_temp[0],lst_temp[1],lst_temp[2]],axis=0)
    X_counts.sort_values(by='mic',inplace=True,ascending=True)
    X_counts['mic'] = X_counts['mic'].astype(str)

    fig = px.bar(X_counts, x='mic',y='counts',color='case')
    fig.update_layout(template='plotly_white',height=300)
    fig.show()

In [ ]:
get_mic()

## 4.4 UNITIG

**UNIQUE VALUES OF ALL UNITIGS**

All columns in the feature matrix .X, used for training contains only values for *present (1)* or not *present (0)* for each Sample_ID.

In [ ]:
column_values = case.X[case.target_name].values.ravel()
print(pd.unique(column_values))

**GROUPED UNITIGS**

Some columns also contain multiple *unitigs* which we can note below as well, perhaps indicating that all grouped unitigs must be present in a given Sample_ID.

In [ ]:
case_unitigs = case.X.columns.tolist()

ii=-1
for i in case_unitigs:
    if(',' in i):
        ii+=1;print(f'{ii} | {i}')

This output shows that some columns represent combined groups of multiple unitigs, and a value of 1 means that all those unitigs are present together in the sample.

**WORKING WITH SEQUENCES**

* Unitigs are composed of **nucleotides**, which means we can use the SQ() class.
* Alternatively, we can use the **BioPython** module as well, storing the sequence data in Seq instances.

In [ ]:
# Using SQ() Class we can define sequences

lst_SQ = []
for unitig in case_unitigs:
    pass
# lst_SQ.append(SQ(unitig,'DNA'))

# print(type(lst_SQ[4]))

# Using BioPython we can define sequences

lst_bSQ = []
for unitig in case_unitigs:
    lst_bSQ.append(Seq(unitig))

print(type(lst_bSQ[4]))

##4.5 Temporal Trends in Antibiotic Resistance

This section explores the temporal patterns of resistance to three antibiotics: **Azithromycin (AZM)**, **Ciprofloxacin (CIP)**, and **Cefixime (CFX)**. By analyzing resistance rates over time, we aim to assess whether resistance to these antibiotics has increased or decreased across the years.


In [ ]:
# Filter out invalid or out-of-range years
metadata_df['Year'] = metadata_df['Year'].astype('Int64')
valid_years_df = metadata_df[(metadata_df['Year'] >= 2000) & (metadata_df['Year'] <= 2025)]

# Group by year and compute resistance percentages
antibiotics = ['azm_sr', 'cip_sr', 'cfx_sr']
resistance_by_year = valid_years_df.groupby('Year')[antibiotics].mean() * 100

# Plot
plt.figure(figsize=(10, 6))
for abx in antibiotics:
    plt.plot(resistance_by_year.index, resistance_by_year[abx], marker='o', label=abx.upper())

plt.title('Antibiotic Resistance Trends Over the Years (AZM, CIP, CFX)', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Resistance Rate (%)')
plt.legend(title='Antibiotic')
plt.grid(True)
plt.tight_layout()
plt.show()


##4.6 Multidrug Resistance (MDR) Profile Distribution

This section investigates the distribution of multidrug resistance (MDR) among the samples based on their resistance to three antibiotics: **Azithromycin (AZM)**, **Ciprofloxacin (CIP)**, and **Cefixime (CFX)**.

A sample is considered **MDR** if it is resistant to **at least two** of these antibiotics. We compute how many samples are resistant to 0, 1, 2, or all 3 of the selected antibiotics and visualize the distribution using a bar plot. This provides an overview of the prevalence of multidrug-resistant profiles in the dataset.


In [ ]:
# Calculate number of resistances per sample across AZM, CIP, CFX
resistance_cols = ['azm_sr', 'cip_sr', 'cfx_sr']
metadata_df['resistance_count'] = metadata_df[resistance_cols].sum(axis=1)

# Count how many samples are resistant to 0, 1, 2, or 3 antibiotics
resistance_distribution = metadata_df['resistance_count'].value_counts().sort_index()

# Plot distribution
plt.figure(figsize=(8, 5))
sns.barplot(x=resistance_distribution.index, y=resistance_distribution.values, palette='Set2')
plt.title('Distribution of Resistance Count (AZM, CIP, CFX)', fontsize=14)
plt.xlabel('Number of Resistant Antibiotics')
plt.ylabel('Number of Samples')
plt.xticks([0, 1, 2, 3])
plt.grid(axis='y')
plt.tight_layout()
plt.show()

##4.7 MIC Distributions and Breakpoints (log2 Transformed)

This section analyzes the distribution of log2-transformed Minimum Inhibitory Concentration (MIC) values for three antibiotics: **Azithromycin (AZM)**, **Ciprofloxacin (CIP)**, and **Cefixime (CFX)**.

MIC distributions provide insights into the population-level susceptibility or resistance tendencies, and when compared against established **clinical breakpoints**, they help distinguish between susceptible and resistant isolates.

Histograms are plotted for each antibiotic’s log2 MIC values, with vertical lines marking the corresponding clinical resistance breakpoints based on EUCAST or CLSI guidelines.


In [ ]:
# Define antibiotics and their log2 MIC columns and breakpoints
mic_info = {
    'AZM': {'col': 'log2_azm_mic', 'breakpoint': 3},   # example: 2^3 = 8 µg/mL
    'CIP': {'col': 'log2_cip_mic', 'breakpoint': -1},  # example: 2^-1 = 0.5 µg/mL
    'CFX': {'col': 'log2_cfx_mic', 'breakpoint': -1}   # example: 2^-1 = 0.5 µg/mL
}

# Plot MIC histograms
plt.figure(figsize=(15, 4))

for i, (abx, info) in enumerate(mic_info.items(), 1):
    plt.subplot(1, 3, i)
    sns.histplot(metadata_df[info['col']].dropna(), bins=20, kde=False, color='steelblue')
    plt.axvline(info['breakpoint'], color='red', linestyle='--', label='Resistance Breakpoint')
    plt.title(f'{abx} - log2 MIC Distribution')
    plt.xlabel('log2(MIC)')
    plt.ylabel('Sample Count')
    plt.legend()

plt.tight_layout()
plt.show()


##4.8 Correlation Between Antibiotics (log2 MIC Values)

In this section, we explore the pairwise correlations between the log2-transformed MIC values of Azithromycin (AZM), Ciprofloxacin (CIP), and Cefixime (CFX).

By computing both the correlation matrix and scatterplots, we aim to visually and statistically assess whether susceptibility to one antibiotic is associated with susceptibility to another. This may help identify shared resistance patterns or independent mechanisms of resistance across the antibiotics.


In [ ]:
# Select log2 MIC columns
mic_cols = ['log2_azm_mic', 'log2_cip_mic', 'log2_cfx_mic']
mic_df = metadata_df[mic_cols].dropna()

# Compute correlation matrix
correlation_matrix = mic_df.corr(method='pearson')

# Display correlation heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title('Correlation Matrix of log2 MIC Values (AZM, CIP, CFX)')
plt.tight_layout()
plt.show()


##4.9 Antibiotic Resistance Trends by Country (Top 5 Sample Contributors)

To investigate potential geographic patterns, we first identify the top 5 countries that contributed the highest number of samples.

We then examine how resistance rates to azithromycin, ciprofloxacin, and cefixime have changed over time within those countries. This analysis may reveal differing national resistance trends and local epidemiological dynamics.


In [ ]:
top_countries = (
    metadata_df['Country']
    .value_counts()
    .head(5)
    .index
    .tolist()
)

filtered_df = metadata_df[metadata_df['Country'].isin(top_countries)]

antibiotics = {
    'azm_sr': 'Azithromycin',
    'cip_sr': 'Ciprofloxacin',
    'cfx_sr': 'Cefixime'
}

for abx_col, abx_name in antibiotics.items():
    trend_df = (
      filtered_df[filtered_df['Year'].between(2000, 2030)]  # Yıl filtresi eklendi
        .groupby(['Country', 'Year'])[abx_col]
        .mean()
        .reset_index()
        .dropna()
    )

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=trend_df, x='Year', y=abx_col, hue='Country', marker='o')
    plt.title(f'Resistance Rate over Time in Top 5 Countries - {abx_name}')
    plt.ylabel('Resistance Rate')
    plt.ylim(0, 1)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

##4.10 Beta-lactamase Gene Presence Over Time

The presence of the Beta-lactamase gene is a critical indicator of bacterial resistance to beta-lactam antibiotics.
This gene can inactivate a wide range of antibiotics, making treatment difficult. In this section, we visualize the
temporal trend of Beta-lactamase gene detection across years to observe any increase or fluctuation over time.


In [ ]:
# Clean and standardize 'Beta.lactamase' values
metadata_df['Beta_lactamase_status'] = metadata_df['Beta.lactamase'].apply(
    lambda x: 'Present' if str(x).strip().upper() in ['1', '2', 'R', 'S'] else 'Absent'
)

# Group by Year and Status
beta_clean_df = (
    metadata_df[metadata_df['Year'].between(1998, 2030)]
    .groupby(['Year', 'Beta_lactamase_status'])['Sample_ID']
    .count()
    .reset_index()
)

# Pivot table for plotting
beta_clean_pivot = beta_clean_df.pivot(index='Year', columns='Beta_lactamase_status', values='Sample_ID').fillna(0)

# Plot
plt.figure(figsize=(10, 5))
beta_clean_pivot.plot(kind='bar', stacked=True, colormap='Paired', figsize=(12, 5))
plt.title('Yearly Trend of Beta-lactamase Presence (Cleaned)')
plt.xlabel('Year')
plt.ylabel('Number of Samples')
plt.legend(title='Beta-lactamase Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

## 4.11 Sample Collection Intensity by Continent and Year

To better understand the structure of the dataset and assess potential sampling bias, we visualized the number of samples collected per continent over time.

This heatmap reveals the temporal and geographic distribution of data collection. It helps identify periods of increased surveillance, under-represented regions, or potential shifts in sampling policies that may affect the interpretation of resistance trends.


In [ ]:
# Group by Continent and Year
collection_df = (
    metadata_df[metadata_df['Year'].between(1990, 2030)]
    .groupby(['Continent', 'Year'])['Sample_ID']
    .count()
    .reset_index()
    .rename(columns={'Sample_ID': 'Sample_Count'})
)

# Pivot for heatmap
heatmap_data = collection_df.pivot(index='Continent', columns='Year', values='Sample_Count').fillna(0)

# Plot heatmap
plt.figure(figsize=(14, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".0f", cmap="YlGnBu", cbar_kws={'label': 'Sample Count'})
plt.title('Sample Collection Intensity by Continent and Year')
plt.xlabel('Year')
plt.ylabel('Continent')
plt.tight_layout()

## 4.12 Beta-lactamase Presence vs Resistance Status Across Antibiotics

To explore the potential association between beta-lactamase enzyme presence and antibiotic resistance, we examined resistance rates (AZM, CIP, CFX) across samples grouped by their beta-lactamase status.

This comparison may reveal whether the production of beta-lactamase — an enzyme capable of breaking down certain antibiotics — correlates with higher resistance rates, even for non-beta-lactam drugs. Although none of the three target antibiotics are beta-lactams, this analysis may indicate co-selection or linked resistance mechanisms.


In [ ]:
# Clean Beta-lactamase values again for consistency
metadata_df['Beta_lactamase_status'] = metadata_df['Beta.lactamase'].apply(
    lambda x: 'Present' if str(x).strip().upper() in ['1', '2', 'R', 'S'] else 'Absent'
)

# Calculate resistance rate per group
resistance_summary = metadata_df.groupby('Beta_lactamase_status')[['azm_sr', 'cip_sr', 'cfx_sr']].mean().reset_index()

# Reshape for plotting
resistance_melted = resistance_summary.melt(id_vars='Beta_lactamase_status',
                                             var_name='Antibiotic',
                                             value_name='Resistance Rate')

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(data=resistance_melted, x='Antibiotic', y='Resistance Rate', hue='Beta_lactamase_status')
plt.title('Antibiotic Resistance by Beta-lactamase Status')
plt.ylabel('Resistance Rate (%)')
plt.xlabel('Antibiotic')
plt.legend(title='Beta-lactamase')
plt.ylim(0, 1)
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', alpha=0.5)

# 5.Creating Resistance Models

## 5.1 List of Tuple Models

**ADDING MODELS TO LIST models**

* We define a list of tuples from which we can access the case (Model Name, Model Instance & CV Parameters) & use them in the evaluation class.
* A dictionary contaning CV Parameters (third term) is only required when calling `.gscv()`

In [ ]:
# models use in kfold
models = []
models.append(
    ('LDA', LinearDiscriminantAnalysis()))

# models use in gscv
cv_lda = {'solver':
              ['svd', 'lsqr', 'eigen']}

models = []
models.append(
    ('LDA', LinearDiscriminantAnalysis(),cv_lda))

## 5.2 Model Evaluation Class

**CLASS INSTANCE INPUTS**

* We'll define a class, class_eval that will be used for evaluation purposes.
* We can instantiate class_eval having defined:
  * *Data/Case Class* containing feature & target vector (get_unitifs or mod_unitigs)
  * *List of models* (containing tuples)
  * *nfold* (number of kfold splits)
  * *gsfold* (number of gridsearch folds)

**EVALUATION OPTIONS**

Using the self.models defined during instantiation:

  * `.cv(type_id)` : kfold Cross validation is done with the selected model(s) with the set hyperparameters.
  * `.gscv(type_id)` : kfold cross validation but with Grid Search, best mean balanced_accuracy score hyperparamers used for fitting on whole kfold subset.

Both functions require kfold type input type_id: Standard (kfold) / Stratified (skfold)

**gscv**

For all input models, defined with grid-search hyperparameters (third term):

  We split the dataset into nfold groups containing both training & test data
  For each train segment data, we use GridSearchCV to find the best scoring model using the provided parameter dictionary.
  Using the best scoring model on the training set, we evaluate both training & test scoring using the evaluation metric.
  For outputs, we recall the best scoring hyperparameters & confusion matrix for each fold.

**cv**

For all input models:

  * We split the dataset into nfold groups containing both training & test data
  * For each nfold, we have a training & test set. On this data we evaluate our model w/ predefined hyperparameters.
  * Results for each fold are stored in .dic_tr & .dic_te which can be visualised by calling .fold_plot()

**EVALUATION METRIC**

* As we saw in Section 2&3, the *target variable* in our binary classification problem can be very one sided
* Let's use *balanced_accuracy*, which is suitable for imbalanced class problems.

In [ ]:
class class_eval():

    def __init__(self,data=None,models=None,nfold = 4,
                 gsfold=3, verbose=True,
                 shuffle_id=False,random_state=22):

        self.shuffle = shuffle_id

        if(self.shuffle is False):
            self.rs = None
        else:
            self.rs = random_state # random state

        if(data is not None):
            X_all = data.X
            self.y = data.X[data.target_name].copy()
            self.X = data.X.drop([data.target_name],axis=1).copy()
        else:
            print('input case data')

        self.models = models # Tuple /w (name,model)
        self.nfold = nfold       # number of cross validation folds
        self.gsfold = gsfold    # number of grid search folds in every fold
        self.target_name = data.target_name  # target name

    ''' Grid-Search Standard Kfold Cross Validation '''
    # For each fold, a grid search cv is applied to find best params

    def gscv(self,type_id='kfold'):

        self.store_models = {} # Store models of Kfolds
        self.dic_tr = {}
        self.dic_te = {}

        for model in self.models:

            if(type_id is 'skfold'):
                kfold = StratifiedKFold(n_splits=self.nfold,
                                        shuffle=self.shuffle,
                                        random_state=self.rs)
            elif(type_id is 'kfold'):
                kfold = KFold(n_splits=self.nfold,
                              shuffle=self.shuffle,
                              random_state=self.rs)

            lst_temp = []; lst_temp2 = []; kfold_id = -1
            for train_index, test_index in kfold.split(self.X,self.y):

                kfold_id+=1;print(f'\nkfold {kfold_id}')
                # split data into train/test sets
                X_train = self.X.iloc[train_index]
                y_train = self.y[train_index]
                X_test = self.X.iloc[test_index]
                y_test = self.y[test_index]

                # perform grid search to identify best hyper-parameters
                gs_clf = GridSearchCV(model[1], param_grid=model[2],
                                      cv=self.gsfold, n_jobs=1,
                                      scoring='balanced_accuracy')

                # Train using the best model
                gs_clf.fit(X_train, y_train)
                best_model = gs_clf.best_estimator_ # best cv model
                self.store_models[f'GS_{model[0]}_{kfold_id}'] = best_model

                # Predict using best model
                ym_tr = gs_clf.predict(X_train)
                ym_te = gs_clf.predict(X_test)
                score_tr = balanced_accuracy_score(y_train,ym_tr)
                score_te = balanced_accuracy_score(y_test,ym_te)
                lst_temp.append(score_tr); lst_temp2.append(score_te)

                print(f'Train Score: {round(score_tr,4)} - Test Score: {round(score_te,4)}')
                print('Best hyperparameters for this fold')
                print(gs_clf.best_params_)
                print(f"Test : Confusion matrix Fold {kfold_id}")
                print(confusion_matrix(y_test, ym_te))

            self.dic_tr[model[0]] = lst_temp
            self.dic_te[model[0]] = lst_temp2
            test_mean = round(sum(lst_temp2)/len(lst_temp2),4)
            print(f'\n {model[0]} - Test Mean Score: {test_mean}')

    '''K-Fold Cross Validation'''
    # w/ type_id option; statified used to make sure classes are balanced in folds

    def cv(self,type_id='kfold'):

        print(f'type_id set to: {type_id}')
        self.store_models = {} # Store models of Kfolds
        self.dic_tr = {}
        self.dic_te = {}

        # Cycle though all tuple model settings
        for model in self.models:

            t0 = time.time()
            if(type_id is 'skfold'):
                kfold = StratifiedKFold(n_splits=self.nfold,
                                        shuffle=self.shuffle,
                                        random_state=self.rs)
            elif(type_id is 'kfold'):
                kfold = KFold(n_splits=self.nfold,
                              shuffle=self.shuffle,
                              random_state=self.rs)

            lst_temp = []; lst_temp2 = []; kfold_id = -1
            for train_index, test_index in kfold.split(self.X,self.y):

                kfold_id+=1
                # split data into train/test sets
                X_train = self.X.iloc[train_index]
                y_train = self.y[train_index]
                X_test = self.X.iloc[test_index]
                y_test = self.y[test_index]

                # Fit, Stopre & Predict Kfold models
                model[1].fit(X_train, y_train)
                self.store_models[f'{model[0]}_{kfold_id}'] = model[1]
                ym_tr = model[1].predict(X_train)
                ym_te = model[1].predict(X_test)

                score_tr = balanced_accuracy_score(y_train,ym_tr)
                score_te = balanced_accuracy_score(y_test,ym_te)
                lst_temp.append(score_tr); lst_temp2.append(score_te)

            self.dic_tr[model[0]] = lst_temp
            self.dic_te[model[0]] = lst_temp2
            test_mean = round(sum(lst_temp2)/len(lst_temp2),4)
            print(f'{model[0]} - Test Mean Score: {test_mean} - Total Time: {round(time.time() - t0,4)}')

    ''' unitig visual functions only '''

    def col_trans(self,sel_id=0):

        # unitig to abrev
        if(sel_id is 0):
            self.X_names = self.X.columns.tolist()
            temp_names = self.X_names.copy()

            unitigs = self.X_names.copy()
            del unitigs[-1]

            lst_abr = []
            ii=-1
            for unitig in range(0,len(unitigs)):
                ii+=1;lst_abr.append(f'u{ii}')
            lst_abr.append(self.target_name)
            self.X.columns = lst_abr
            self.dicabr = dict(zip(lst_abr,self.X_names))

        # abreb to unitig
        elif(sel_id is 1):
            self.X.columns = self.X_names

    ''' PLOT KFOLD RESULTS '''

    def fold_plot(self):

        df_tr = pd.DataFrame(self.dic_tr)
        df_te = pd.DataFrame(self.dic_te)

        # Make Plots
        fig,ax = plt.subplots(2,2,figsize=(14,6))
        sns.stripplot(data=df_tr, orient='h',linewidth=1,ax=ax[0,0])
        sns.heatmap(data=df_tr,annot=True,cbar=False,cmap="plasma_r",
                    vmax=1,vmin=0.5,ax=ax[0,1])
        sns.stripplot(data=df_te, orient='h',linewidth=1,ax=ax[1,0])
        sns.heatmap(data=df_te,annot=True,cbar=False,cmap="plasma_r",
                    vmax=1,vmin=0.5,fmt='.3g',ax=ax[1,1])
        ax[0,0].set_xlim([0.5,1.1]);ax[1,0].set_xlim([0.5,1.1])
        ax[0,0].set_title('train kfold')
        ax[0,1].set_title('train kfold heatmap')
        ax[1,0].set_title('test kfold')
        ax[1,1].set_title('test kfold heatmap')
        sns.despine(bottom=True, left=True)
        plt.tight_layout()

# 6.Ciprofloxacin Resistance Models

## 6.1.Model Summary

* *Kfold Resistance Models* - Set benchmark model scores, outline promising models, quite different models are chosen to see which approaches work best for this problem
* *GridSearchCV Resistance Models* - As one of the more well performing models that has feature importance as well, Let's try to improve the CatBoost model accuracy & check how kfold models differ when it comes to feature importance
* *SVC Based Resistance Models* - SVC offers feature importance similar to LinearRegression(), we'll be able to know which top features favours which particular classes in terms of weighting, so if we can make sure the model performs well, we'll have some more unique insight on top of the tree based approach, so we'll try to tune the model so its as accurate as it can be so we have more confidence in the result

## 6.2 K-FOLD Resistance Models

**CHOOSING MODELS FOR EVALUATION**

* We add models we would like to use for cross validation both unsupervised & supervised learning models.
* Hyperparameter selections are quite arbitrary, tree based methods use n_estimators = 10, which doesn't seem like a bad place to start.

In [ ]:
''' Define Models used for Testing '''
models = []
n_est = 10

# Unsupervised Learning Models
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))

# # Supervised Learning Models
models.append(('SVC',SVC()))
models.append(('TREE', DecisionTreeClassifier())) # Supervised Model
models.append(('GBM', GradientBoostingClassifier(n_estimators=n_est)))

# Desirable for Feature Importance Evaluation
models.append(('XGB',XGBClassifier(n_estimators=n_est,verbosity = 0)))
models.append(('CAT',CatBoostClassifier(silent=True,n_estimators=n_est)))
models.append(('RF', RandomForestClassifier(n_estimators=n_est)))

**CASE INPUT**

* Using the unmodified feature matrix of `get_unitigs().get_case()`
* Using a standard 4-fold cross validation strategy, let's see how well the models perfom on each fold.

In [ ]:
# Get case; (Dataset Feature Class)
case = get_unitigs()
case.get_case('cip_sr')

# Standard KFOLD evaluation
eval1 = class_eval(data=case, # input the case class
                   nfold = 4, # 4 fold kfold
                   models=models) # global models tuple list)

# Evaluate kfold using selected models
eval1.cv(type_id='kfold')  # standard kfold evaluation

In [ ]:
eval1.fold_plot()  # plot kfold results

For predicting ciprofloxacin resistance, the most reliable and consistent models are CatBoost (CAT), XGBoost (XGB), Gradient Boosting (GBM), and Random Forest (RF), all showing high and stable performance across test folds. In contrast, LDA performs poorly with low and unstable test scores, making it unsuitable for this task. Models like Naive Bayes (NB) also show variance between folds, indicating a potential need for hyperparameter tuning or regularization to improve generalization.

**TREE BASED FEATURE IMPORTANCE**

* Using *RandomForest*,*CatBoost*,*XGBRegressor* we have access to the model's relative feature importance
* It's useful to compare the feature importance of multiple models together. We have over 8000 unitigs; let's pick the top 10 most influential.
* The most critical unitigs to the determination of antibiotic resistance (according to the model) are saved in BioPython Seq format & can be used for further analyses.

In [ ]:
''' Tree Based Feature Importance '''
# requires evaluation class input w/ at least one of RF, CatBoost & XGB moedls
# models stored in .store_models are required from eval class

class fi:

    def __init__(self,data=None, # evaluation class
                      sort_by='RF', # show most important features
                      max_features=10 # limit unitigs to
                ):

        if(data is None):
            print('Enter Evaluation class w/ CAT,RF,XGB')
        else:
            evals = data
            # check which models are present
            lst_models = list(evals.store_models.keys())
            temp = []
            for i in lst_models:
                if('CAT' in i):
                    temp.append('CAT')
                if('RF' in i):
                    temp.append('RF')
                if('XGB' in i):
                    temp.append('XGB')

            # input contains gscv data
            if('GS' in lst_models[0]):
                self.gs_id = True
            else:
                self.gs_id = False

        self.lst_tree_models = list(set(temp))

        self.evals = data  # evaluation class
        self.lst_Seqs = []  # list of important unitigs
        self.max_features = max_features # show top n features
        self.sort_by = sort_by # sort by particualr model fi, other mods show this index only
        self.abr_feat = False # activate if unitig names are too big for figure

    # Compile all Tree based feature importance results
    def get(self):

        # USL scaling
        min_max_scaler = preprocessing.MinMaxScaler()

        # Recall Model & Get Feature Importance from data class
        # unless gridsearched, all kfolds are the same model

        ii=-1
        # if randomforest models are present
        if('RF' in self.lst_tree_models):

            if(self.gs_id):

                # fold names
                tlst_models = [f'GS_RF_{i}' for i in range(0,self.evals.nfold)]

                # stack all fold results
                for kfold_id in tlst_models:
                    ii+=1
                    rf_model = self.evals.store_models[kfold_id]
                    imp_rf = rf_model.feature_importances_
                    rf_sc = min_max_scaler.fit_transform(imp_rf[:,None])
                    ldf = pd.DataFrame(rf_sc,index=self.evals.X.columns,columns=[kfold_id])
                    if(ii is 0):
                        df = ldf.copy()
                    else:
                        df = pd.concat([df,ldf],axis=1)

            else:

                ii+=1
                rf_model = self.evals.store_models['RF_1']
                imp_rf = rf_model.feature_importances_
                rf_sc = min_max_scaler.fit_transform(imp_rf[:,None])
                ldf = pd.DataFrame(rf_sc,index=self.evals.X.columns,columns=['RF'])
                if(ii is 0):
                    df = ldf.copy()
                else:
                    df = pd.concat([df,ldf],axis=1)

        # if catboost models are present
        if('CAT' in self.lst_tree_models):

            if(self.gs_id):

                # fold names
                tlst_models = [f'GS_CAT_{i}' for i in range(0,self.evals.nfold)]

                # stack all fold results
                for kfold_id in tlst_models:
                    ii+=1
                    cb_model = self.evals.store_models[kfold_id]
                    imp_cb = cb_model.get_feature_importance()
                    cb_sc = min_max_scaler.fit_transform(imp_cb[:,None])
                    ldf = pd.DataFrame(cb_sc,index=self.evals.X.columns,columns=[kfold_id])
                    if(ii is 0):
                        df = ldf.copy()
                    else:
                        df = pd.concat([df,ldf],axis=1)

            else:
                ii+=1
                cb_model = self.evals.store_models['CAT_1']
                imp_cb = cb_model.get_feature_importance()
                cb_sc = min_max_scaler.fit_transform(imp_cb[:,None])
                ldf = pd.DataFrame(cb_sc,index=self.evals.X.columns,columns=['CB'])
                if(ii is 0):
                    df = ldf.copy()
                else:
                    df = pd.concat([df,ldf],axis=1)


        if('XGB' in self.lst_tree_models):

            if(self.gs_id):

                # fold names
                tlst_models = [f'GS_XGB_{i}' for i in range(0,self.evals.nfold)]

                # stack all fold results
                for kfold_id in tlst_models:
                    ii+=1
                    xg_model = self.evals.store_models[kfold_id]
                    imp_xg = xg_model.feature_importances_
                    xg_sc = min_max_scaler.fit_transform(imp_xg[:,None])
                    ldf = pd.DataFrame(xg_sc,index=self.evals.X.columns,columns=[kfold_id])
                    if(ii is 0):
                        df = ldf.copy()
                    else:
                        df = pd.concat([df,ldf],axis=1)

            else:

                ii+=1
                xg_model = self.evals.store_models['XGB_1']
                imp_xg = xg_model.feature_importances_
                xg_sc = min_max_scaler.fit_transform(imp_xg[:,None])
                ldf = pd.DataFrame(rf_sc,index=self.evals.X.columns,columns=['XGB'])

                if(ii is 0):
                    df = ldf.copy()
                else:
                    df = pd.concat([df,ldf],axis=1)

        # change to abbrev if names are too long to display
        if(self.abr_feat):
            self.evals.col_trans(0)

        # Sort by one of the available columns
        df.sort_values(by=self.sort_by,ascending=False,inplace=True)

        if(self.abr_feat):
                self.evals.col_trans(1)

        # show only most critical features in FI
        subset = df[:self.max_features]

#       Store the most important features
        for i in subset.index.tolist():
            self.lst_Seqs.append(Seq(i))

        # Plot features
        fig = px.bar(subset,orientation='h')
        fig.update_traces(width=0.5)
        fig.update_layout(height=400,template='plotly_white',
                          title=f"<b>FEATURE IMPORTANCE</b> | Sorted by {self.sort_by}()")
        fig.show()

In [ ]:
crit_unitigs = fi(data=eval1,sort_by='CB')
crit_unitigs.get()

In [ ]:
crit_unitigs.lst_Seqs

## 6.3. GridSearchCV Resistance Model

**CATBOOST HYPERPARAMETER TUNING**

* We saw that *CatboostClassifier* had already has quite good results on the test set, let's try to optimise the hyperparameters, if we can.
* Catboost does optimise certain parameters that aren't self defined, so we may not get any improvement, however we should get different hyerparameter combinations on different folds, which will be handy to check if there is any feature importance variation.

**GETTING ALL MODELS PARAMETERS**

* We can call `.get_all_params()` to display all the parameters that were set in CatBoost, on top of the n_estimators that we set ourselves.
* The learning rate & n_estimators/iterations are two hyperparameters we can try to tune.

In [ ]:
# Show default parameters used in catboost model
eval1.store_models['CAT_0'].get_all_params()

DEFINING GRID & GRIDSEARCHCV

* We define a parameter grid, which will be used in GridSearchCV; keeping it simple, looking at only n_estimators (iters) & the learning rate.
* Defining a standard 4-fold cross validation strategy once again, we should get 4 slightly different models this time, which are stored in .store_models().
* Catboost is of course compatible with the GPU, so we can set task_type = "GPU".

In [ ]:
# Define Grid used in Cross Validation
params = {'n_estimators':[10,25,40],
         'learning_rate':[0.01,0.05,0.5]}

# Define Model (just the one)
models = []
models.append(('CAT', CatBoostClassifier(silent=True,
                                         task_type="GPU"),params))

# Get Dataset Features
case2 = get_unitigs()
case2.get_case('cip_sr')

eval2 = class_eval(data=case2,
                   models=models)
eval2.gscv()

**PLOTTING KFOLD MODEL FEATURE IMPORTANCES**

* Passing on an eval class instance into class fi, we have access to store_models, which when using the gridsearchcv option in class eval stores the fold models.
* Unlike .cv(), we probably will have slightly different model hyperparameters for each fold, feature importance of these models can also be visualised.

In [ ]:
crit_unitigs = fi(data=eval2,sort_by='GS_CAT_0')
crit_unitigs.get()

In [ ]:
crit_unitigs.lst_Seqs

**REVISITING UNITIG DISTRIBUTIONS**

* Having identified the key features (unitigs) which have the most weight in its respective model
* With tree based model feature importance, we ultimately get only positive values & don't have an indication about which class the result is leaning towards
* However knowing which features features (unitigs) to look at now, we can use group_by to get some idea since they are all of One Hot Encoding type

In [ ]:
case = get_unitigs()
case.get_case('cip_sr')

print(case.X.groupby('cip_sr')['GTGCGACAGCAAAGTCCAAACCAGCGTCCCCGCC'].mean())
print(case.X.groupby('cip_sr')['GCGCAGCCGCAAATCTTGTTTTCCCATTCCGCC'].mean())
print(case.X.groupby('cip_sr')['AAATTGCGGATCGATGCGCGAAGGGTCGAATGC'].mean())

print(case.X.groupby('cip_sr')['GGCATCCCGAAGCCGAATACGGCAACGGCAAGCG'].mean())

## 6.4 SVC Model Resistance Models

**SVC SUBOPTIMAL MODEL**

* In previous section, we used the SVC model, using the default hyperparameters, which uses kernel='rbf'
* SVC gives us the option to evaluate the feature importance of weights, if we use kernel='linear'
* Unlike the tree based feature importance, we will know towards which class the particular feature leans towards (importance wise)
* Let's try a default model first, followed by a gridsearchcv optimised model, so we can see if there is any difference in feature importance

In [ ]:
''' Define Models used for Testing '''
models = []
models.append(('SVC',SVC(kernel = 'linear')))

# Get case; (Dataset Feature Class)
case = get_unitigs()
case.get_case('cip_sr')

# Standard KFOLD evaluation
eval_svc1 = class_eval(data=case, # input the case class
                   nfold = 4, # 4 fold kfold
                   models=models) # global models tuple list)

# Evaluate kfold using selected models
eval_svc1.cv(type_id='kfold')  # standard kfold evaluation

In [ ]:
''' Function plots & returns highest weighted features '''
# for SVC linear covariance function model in SQ sequence format

def fi_svc(classifier, feature_names, top_features=5,verbose=False):

    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])

    # plt.title("Feature Importances (Support Vector Machine) - Ciprofloxacin Resistance", y=1.08)
    colors = ['crimson' if c < 0 else 'cornflowerblue' for c in coef[top_coefficients]]
    feature_names = np.array(feature_names)
    lser = pd.Series(data=coef[top_coefficients],index=feature_names[top_coefficients])
    fig = px.bar(lser,orientation='h')
    fig.update_traces(width=0.5)
    fig.update_layout(height=350,template='plotly_white',showlegend=False,
                        title=f"<b>FEATURE IMPORTANCE</b> | SVC")
    fig.show()

    # if we print the unitigs, we can then look at what genes they relate to
    top_negative_coefficients = np.argsort(coef)[:5]
    neg_predictors = np.asarray(feature_names)[top_negative_coefficients]
    top_positive_coefficients = np.argsort(coef)[-5:]
    pos_predictors = np.asarray(feature_names)[top_positive_coefficients]
    if(verbose):
        print("Top negative predictors: ",neg_predictors)
        print("Top positive predictors: ",pos_predictors)

    # Store the most important features
    top_negSeq = []; top_posSeq = []
    for i in range(0,top_features):
        top_negSeq.append(Seq(neg_predictors[i]))
        top_posSeq.append(Seq(pos_predictors[i]))

    return top_negSeq, top_posSeq

In [ ]:
eval_svc1.store_models

In [ ]:
neg_predictors,pos_predictors = fi_svc(eval_svc1.store_models['SVC_3'], list(eval_svc1.X.columns))

In [ ]:
# return list of sequences list ordered from most to least important
neg_predictors

In [ ]:
# return list of sequences ordered from least to most important
pos_predictors

**SVC OPTIMISED MODEL**

* The linear covariance function model performed a little worse than those tested in the previous section
* Let's see if we can optimise the model a little bit & check if the same features will remain the most important in the model
* Let's try a variation in the gamma hyperparameter for the search grid to keep things quite simple

In [ ]:
svm_params = {
    'C': [0.01],
    'gamma': [1e-06, 1e-05],
    'kernel': ['linear']
}

# Define Model (just the one)
svm = SVC(class_weight='balanced')
models = []
models.append(('SVM',svm,svm_params))

# Get Dataset Features
case = get_unitigs()
case.get_case('cip_sr')

eval_svc2 = class_eval(data=case,
                   models=models)
eval_svc2.gscv()

In [ ]:
eval_svc2.store_models

In [ ]:
neg_predictors,pos_predictors = fi_svc(eval_svc2.store_models['GS_SVM_0'], list(eval_svc2.X.columns))

In [ ]:
# return list of sequences list ordered from most to least important
neg_predictors

In [ ]:
# return list of sequences ordered from least to most important
pos_predictors

## 6.5 Cefixime Resistance – Model Evaluation Summary


In [ ]:
# 1. K-Fold sonuçlarını DataFrame’e çevir
df_tr_cip = pd.DataFrame(eval1.dic_tr)
df_te_cip = pd.DataFrame(eval1.dic_te)

# 2. Ortalama skorları hesapla
summary_cip = pd.DataFrame({
    "Model": df_te_cip.columns,
    "Train Mean Balanced Accuracy": df_tr_cip.mean().values,
    "Test Mean Balanced Accuracy": df_te_cip.mean().values
}).sort_values("Test Mean Balanced Accuracy", ascending=False)

# 3. Tabloyu göster
display(summary_cip)

# 4. Grafikle görselleştir
plt.figure(figsize=(10, 6))
sns.barplot(data=summary_cip, x="Test Mean Balanced Accuracy", y="Model", palette="Oranges_d")
plt.title("Classifier Performance on Ciprofloxacin Resistance (Test Balanced Accuracy)", fontsize=14)
plt.xlabel("Mean Balanced Accuracy (Test)")
plt.ylabel("Model")
plt.xlim(0.5, 1.0)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## 6.6 Ciprofloxacin Resistance – SHAP Explainability (CAT Model)

The CatBoost classifier, which achieved the highest test balanced accuracy (0.9694) for Ciprofloxacin resistance prediction, was further analyzed using SHAP (SHapley Additive exPlanations) to interpret feature contributions.

The **global explainability** analysis, presented through SHAP summary and bar plots, highlights the most influential genomic unitigs contributing to resistance classification. These insights help identify the genomic signatures most associated with Ciprofloxacin resistance.

Additionally, **local explanations** illustrate how specific features drive individual predictions, providing a clearer understanding of the classifier’s decision-making process for selected cases.


In [ ]:
# 1) Choosing the CAT Model which is Higher Score
cat_keys = [k for k in getattr(eval1, "store_models", {}).keys() if k.startswith("GS_CAT_")] \
           or [k for k in getattr(eval1, "store_models", {}).keys() if k.startswith("CAT_")]
if not cat_keys:
    raise RuntimeError("CatBoost modeli bulunamadı. Lütfen Ciprofloxacin için CAT modeli eğitildiğinden emin olun.")

cat_key = sorted(cat_keys)[0]
cat_model = eval1.store_models[cat_key]

# 2) Feature Matrix
X_cip = eval1.X.drop(columns=[eval1.target_name], errors="ignore")

# 3) TreeExplainer + SHAP Values
explainer = shap.TreeExplainer(cat_model)
shap_values = explainer.shap_values(X_cip)

# 4) Beeswarm
plt.figure(figsize=(10, 6))
try:
    shap.summary_plot(shap_values[1], X_cip, show=False)  # ikili sınıfta pozitif sınıf
except Exception:
    shap.summary_plot(shap_values, X_cip, show=False)
plt.title("SHAP Summary – Ciprofloxacin / CAT", fontsize=14)
plt.show()

print("---------------------------------")
print("---------------------------------")
print("---------------------------------")
print("---------------------------------")

# 5) Bar (mean |SHAP|)
plt.figure(figsize=(10, 6))
try:
    shap.summary_plot(shap_values[1], X_cip, plot_type="bar", show=False)
except Exception:
    shap.summary_plot(shap_values, X_cip, plot_type="bar", show=False)
plt.title("Mean |SHAP| – Ciprofloxacin / CAT", fontsize=14)
plt.show()

In [ ]:
# Optional: Top 10 Features Table
try:
    sv = shap_values[1] if isinstance(shap_values, list) else shap_values
    imp = np.abs(sv).mean(axis=0)
    top10 = pd.DataFrame({"feature": X_cip.columns, "mean_abs_shap": imp}).sort_values(
        "mean_abs_shap", ascending=False
    ).head(10).reset_index(drop=True)
    display(top10)
except Exception as e:
    print("Top10 tablo üretilemedi:", e)

## 6.7 Cefixime Resistance – LIME Explainability (GBM Model)
In this section, we interpret the best-performing Cefixime model (GBM) using LIME (model-agnostic, local explanations). The procedure selects the trained GBM from the evaluation pipeline, builds a tabular explainer on the feature matrix (target removed), and explains a chosen instance (default: index 0). We visualize the weighted feature contributions and list the top-k influential unitigs for that prediction.

In [ ]:
# 1) Pick the CAT model key (prefer GS_CAT_* over CAT_*)
cat_keys = [k for k in getattr(eval1, "store_models", {}).keys() if k.startswith("GS_CAT_")] \
           or [k for k in getattr(eval1, "store_models", {}).keys() if k.startswith("CAT_")]
if not cat_keys:
    raise RuntimeError("No CatBoost model found for Ciprofloxacin. Please run the CAT training cells first.")
cat_key   = sorted(cat_keys)[0]
cat_model = eval1.store_models[cat_key]

# 2) Build the feature matrix (drop target) and class names
X_cip = eval1.X.drop(columns=[eval1.target_name], errors="ignore").copy()
y_cip = getattr(eval1, "y", None)
class_names = ["0","1"] if y_cip is None else [str(c) for c in sorted(pd.unique(y_cip))]

# 3) Create a LIME tabular explainer on the training data
explainer = LimeTabularExplainer(
    training_data=X_cip.values,
    feature_names=X_cip.columns.tolist(),
    class_names=class_names,
    mode="classification",
    discretize_continuous=True
)

# 4) Choose an instance to explain and define predict_fn
instance_idx = 0          # change as needed (e.g., 10, 25, ...)
num_features = 10         # number of top contributing features to list
num_samples  = 5000       # number of synthetic samples LIME will generate

x0 = X_cip.iloc[instance_idx].values
predict_fn = lambda data: cat_model.predict_proba(pd.DataFrame(data, columns=X_cip.columns))

# 5) Generate the explanation and visualize it
exp = explainer.explain_instance(
    data_row=x0,
    predict_fn=predict_fn,
    num_features=num_features,
    num_samples=num_samples
)

fig = exp.as_pyplot_figure()
_ = fig.suptitle(f"LIME Explanation – Ciprofloxacin / CAT – {cat_key} – idx={instance_idx}", y=1.02)
plt.show()

# 6) Human-readable list (feature rule, weight) + DataFrame
pairs = exp.as_list()  # [(feature_rule, weight), ...]
print(f"\nTop {num_features} features (LIME weights):")
for f, w in pairs:
    print(f"{f}  ==>  {w:+.4f}")

top_df = pd.DataFrame(pairs, columns=["feature_rule", "lime_weight"])
display(top_df)

# 7.Azithromycin Resistance Models

## 7.1 Load and summarise azm_sr dataset

In [ ]:
case_azm = get_unitigs()
case_azm.get_case('azm_sr')
print(case_azm.X[case_azm.target_name].value_counts())
print("Shape:", case_azm.X.shape)

## 7.2 Class re-balancing with SMOTENC

In [ ]:
mod_azm = mod_unitigs(case_azm)
mod_azm.smote(smote_id='smotenc',
              smote_strat=0.5,
              k_neighbours=5)
print(mod_azm.X[mod_azm.target_name].value_counts())

## 7.3 Model roster & 4-fold CV

In [ ]:
models_azm = []
n_est = 10
models_azm += [
    ('LDA', LinearDiscriminantAnalysis()),
    ('KNN', KNeighborsClassifier()),
    ('NB',  GaussianNB()),
    ('SVC', SVC()),
    ('TREE', DecisionTreeClassifier()),
    ('GBM', GradientBoostingClassifier(n_estimators=n_est)),
    ('XGB', XGBClassifier(n_estimators=n_est, verbosity=0)),
    ('CAT', CatBoostClassifier(silent=True, n_estimators=n_est)),
    ('RF',  RandomForestClassifier(n_estimators=n_est))
]
eval_azm = class_eval(data=mod_azm, models=models_azm, nfold=4)
eval_azm.cv(type_id='kfold')

## 7.5 SVC GridSearch + coef analysis

In [ ]:
svc_params = {'C':[0.01,0.1,1,10],
              'gamma':[1e-4,1e-5],
              'kernel':['linear']}
gs_svc = [('SVC', SVC(class_weight='balanced'), svc_params)]
eval_azm_svc = class_eval(data=mod_azm, models=gs_svc, nfold=4,
                          shuffle_id=True, random_state=42)
eval_azm_svc.gscv()

## 7.6 Azithromycin Feature importance

In [ ]:
neg_seq_azm, pos_seq_azm = fi_svc(
    eval_azm_svc.store_models['GS_SVC_0'],
    list(eval_azm_svc.X.columns),
    top_features=5, verbose=True)

In [ ]:
neg_seq_azm

In [ ]:
pos_seq_azm

## 7.7 Azithromycin Resistance – Model Evaluation Summary


In [ ]:
# 1. Skor sözlüklerini DataFrame’e çevir
df_tr_azm = pd.DataFrame(eval_azm.dic_tr)
df_te_azm = pd.DataFrame(eval_azm.dic_te)

# 2. Ortalama skorları hesapla
summary_azm = pd.DataFrame({
    "Model": df_te_azm.columns,
    "Train Mean Balanced Accuracy": df_tr_azm.mean().values,
    "Test Mean Balanced Accuracy": df_te_azm.mean().values
}).sort_values("Test Mean Balanced Accuracy", ascending=False)

# 3. Tabloyu göster
display(summary_azm)

# 4. Grafikle görselleştir
plt.figure(figsize=(10, 6))
sns.barplot(data=summary_azm, x="Test Mean Balanced Accuracy", y="Model", palette="Greens_d")
plt.title("Classifier Performance on Azithromycin Resistance (Test Balanced Accuracy)", fontsize=14)
plt.xlabel("Mean Balanced Accuracy (Test)")
plt.ylabel("Model")
plt.xlim(0.5, 1.0)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 7.8 Azithromycin SHAP Explainability (LDA Model)

In this section, we interpret the best-performing model for Azithromycin resistance prediction using SHAP.  
From the model evaluation results, **Linear Discriminant Analysis (LDA)** achieved the highest test mean balanced accuracy (0.8412), outperforming other algorithms.  
Since LDA is not tree-based, we use **SHAP’s model-agnostic KernelExplainer** to compute feature contributions.

In [ ]:
# 1) LDA modelini eval_azm içinden al
lda_keys = [k for k in getattr(eval_azm, "store_models", {}).keys() if k.startswith("LDA_") or k.startswith("GS_LDA_")]
if not lda_keys:
    raise RuntimeError("Azithromycin için LDA modeli bulunamadı; önce ilgili model hücrelerini çalıştırın.")
lda_key   = sorted(lda_keys)[0]
lda_model = eval_azm.store_models[lda_key]

# 2) Özellik matrisi (hedefi çıkar)
X_all = eval_azm.X.drop(columns=[eval_azm.target_name], errors="ignore")

# 3) Arka plan (background) ve açıklanacak örneklem (hız için sınırlıyoruz)
#    Not: k-means merkezleri genelde daha hızlıdır; istersen sample() da kullanabilirsin.
X_bg     = shap.kmeans(X_all, 50)                # ~50 merkez (hız/kararlılık dengesi)
X_sample = X_all.sample(min(len(X_all), 300), random_state=42)

# 4) Model çıktısını KernelExplainer’a uygun fonksiyon olarak tanımla
#    Tercihen predict_proba'nın pozitif sınıf (1) olasılığını döndürelim.
def f_proba(data):
    df = pd.DataFrame(data, columns=X_all.columns)
    # Çoğu sklearn LDA'da predict_proba vardır; yoksa decision_function kullan.
    if hasattr(lda_model, "predict_proba"):
        return lda_model.predict_proba(df)[:, 1]
    else:
        # decision_function dönerse onu sigmoid'e map etmek seçenek olabilir,
        # fakat çoğu LDA'da predict_proba mevcut.
        return lda_model.decision_function(df)

# 5) KernelExplainer ve SHAP değerleri
explainer   = shap.KernelExplainer(f_proba, X_bg)
shap_values = explainer.shap_values(X_sample, nsamples=100)   # nsamples↑ = daha doğru, ama yavaş

# 6) Global açıklama – Beeswarm
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_sample, show=False)
plt.title("SHAP Summary – Azithromycin / LDA (KernelExplainer)", fontsize=14)
plt.show()

print("----------------------")
print("----------------------")
print("----------------------")

# 7) Global açıklama – Bar (mean |SHAP|)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
plt.title("Mean |SHAP| – Azithromycin / LDA (KernelExplainer)", fontsize=14)
plt.show()

In [ ]:
# 8) Optional Top-10 features
imp = np.abs(shap_values).mean(axis=0)
top10_azm = pd.DataFrame({"feature": X_sample.columns, "mean_abs_shap": imp}) \
              .sort_values("mean_abs_shap", ascending=False) \
              .head(10).reset_index(drop=True)
display(top10_azm)

## 7.9 Azithromycin Resistance – LIME Explainability (LDA Model)

In this section, we explain the best-performing Azithromycin model (LDA) using LIME (local, model-agnostic). The code below selects the trained LDA from the evaluation pipeline, builds a tabular explainer on the feature matrix (target removed), chooses an instance to explain, and then produces (i) a local bar plot of weighted feature contributions and (ii) a readable top-k feature list with weights.

In [ ]:
# 1) Pick the LDA model key (prefer GS_LDA_* over LDA_*)
lda_keys = [k for k in getattr(eval_azm, "store_models", {}).keys() if k.startswith("GS_LDA_")] \
           or [k for k in getattr(eval_azm, "store_models", {}).keys() if k.startswith("LDA_")]
if not lda_keys:
    raise RuntimeError("No LDA model found for Azithromycin. Please run the LDA training cells first.")
lda_key   = sorted(lda_keys)[0]
lda_model = eval_azm.store_models[lda_key]

# 2) Build the feature matrix (drop target) and class names
X_azm = eval_azm.X.drop(columns=[eval_azm.target_name], errors="ignore").copy()
y_azm = getattr(eval_azm, "y", None)
class_names = ["0","1"] if y_azm is None else [str(c) for c in sorted(pd.unique(y_azm))]

# 3) Create a LIME tabular explainer using the training matrix
explainer = LimeTabularExplainer(
    training_data=X_azm.values,
    feature_names=X_azm.columns.tolist(),
    class_names=class_names,
    mode="classification",
    discretize_continuous=True
)

# 4) Choose an instance to explain and define predict_fn
instance_idx = 0          # change to 10, 25, ... as needed
num_features = 10         # number of top contributing features to list
num_samples  = 5000       # number of synthetic samples LIME will generate

x0 = X_azm.iloc[instance_idx].values

# LDA usually supports predict_proba; fallback to decision_function if needed
def predict_fn(data):
    df = pd.DataFrame(data, columns=X_azm.columns)
    if hasattr(lda_model, "predict_proba"):
        return lda_model.predict_proba(df)
    else:
        # If only decision_function exists, convert to a 2-column "prob-like" output
        # (simple logistic mapping for demonstration; adjust if your API differs)
        scores = lda_model.decision_function(df)
        # Map scores to (p0, p1)
        p1 = 1 / (1 + np.exp(-scores))
        p0 = 1 - p1
        return np.c_[p0, p1]

# 5) Generate the explanation and visualize it
exp = explainer.explain_instance(
    data_row=x0,
    predict_fn=predict_fn,
    num_features=num_features,
    num_samples=num_samples
)

fig = exp.as_pyplot_figure()
_ = fig.suptitle(f"LIME Explanation – Azithromycin / LDA – {lda_key} – idx={instance_idx}", y=1.02)
plt.show()

# 6) Human-readable list (feature rule, weight) + DataFrame
pairs = exp.as_list()  # [(feature_rule, weight), ...]
print(f"\nTop {num_features} features (LIME weights):")
for f, w in pairs:
    print(f"{f}  ==>  {w:+.4f}")

top_df = pd.DataFrame(pairs, columns=["feature_rule", "lime_weight"])
display(top_df)

# 8.Cefixime Resistance Models

## 8.1 Load and summarise cfx_sr dataset

In [ ]:
case_cfx = get_unitigs()
case_cfx.get_case('cfx_sr')
print(case_cfx.X[case_cfx.target_name].value_counts())
print("Shape:", case_cfx.X.shape)

## 8.2 Down-sample + SMOTENC balancing

In [ ]:
mod_cfx = mod_unitigs(case_cfx)
mod_cfx.train  = mod_cfx.X.drop([mod_cfx.target_name], axis=1)
mod_cfx.target = mod_cfx.X[mod_cfx.target_name].copy()
mod_cfx.split_case(frac_id=0.1)         # keep 10 % of class 0
mod_cfx.smote(smote_id='smotenc',
              smote_strat=1.0,
              k_neighbours=2)
print(mod_cfx.X[mod_cfx.target_name].value_counts())

## 8.3 Model roster & 4-fold CV

In [ ]:
models_cfx = []
models_cfx += [
    ('LDA',  LinearDiscriminantAnalysis()),
    ('KNN',  KNeighborsClassifier()),
    ('NB',   GaussianNB()),
    ('SVC',  SVC()),
    ('TREE', DecisionTreeClassifier()),
    ('GBM',  GradientBoostingClassifier(n_estimators=10)),
    ('XGB',  XGBClassifier(n_estimators=10, verbosity=0)),
    ('CAT',  CatBoostClassifier(silent=True, n_estimators=10)),
    ('RF',   RandomForestClassifier(n_estimators=10))
]
eval_cfx = class_eval(data=mod_cfx, models=models_cfx, nfold=4)
eval_cfx.cv(type_id='kfold')

## 8.4 GBM GridSearch

In [ ]:
gbm_params = {'n_estimators':[50,100,150],
              'learning_rate':[0.01,0.05,0.1],
              'max_depth':[3,4,5]}
gs_gbm = [('GBM', GradientBoostingClassifier(), gbm_params)]
eval_cfx_gbm = class_eval(data=mod_cfx, models=gs_gbm, nfold=4,
                          shuffle_id=True, random_state=42)
eval_cfx_gbm.gscv()

## 8.5 SVC GridSearch + coef analysis

In [ ]:
svc_params = {'C':[0.01,0.1,1,10],
              'gamma':[1e-4,1e-5],
              'kernel':['linear']}
gs_svc_cfx = [('SVC', SVC(class_weight='balanced'), svc_params)]
eval_cfx_svc = class_eval(data=mod_cfx, models=gs_svc_cfx, nfold=4,
                          shuffle_id=True, random_state=42)
eval_cfx_svc.gscv()


## 8.6 Cefixime Feature Importance

In [ ]:
neg_seq_cfx, pos_seq_cfx = fi_svc(
    eval_cfx_svc.store_models['GS_SVC_0'],
    list(eval_cfx_svc.X.columns),
    top_features=5, verbose=True)

In [ ]:
neg_seq_cfx

In [ ]:
pos_seq_cfx

## 8.7 Cefixime Resistance – Model Evaluation Summary


In [ ]:
# 1. Skor sözlüklerini DataFrame’e çevir
df_tr_cfx = pd.DataFrame(eval_cfx.dic_tr)
df_te_cfx = pd.DataFrame(eval_cfx.dic_te)

# 2. Ortalama skorları hesapla
summary_cfx = pd.DataFrame({
    "Model": df_te_cfx.columns,
    "Train Mean Balanced Accuracy": df_tr_cfx.mean().values,
    "Test Mean Balanced Accuracy": df_te_cfx.mean().values
}).sort_values("Test Mean Balanced Accuracy", ascending=False)

# 3. Tabloyu göster
display(summary_cfx)

# 4. Grafikle görselleştir
plt.figure(figsize=(10, 6))
sns.barplot(data=summary_cfx, x="Test Mean Balanced Accuracy", y="Model", palette="Blues_d")
plt.title("Classifier Performance on Cefixime Resistance (Test Balanced Accuracy)", fontsize=14)
plt.xlabel("Mean Balanced Accuracy (Test)")
plt.ylabel("Model")
plt.xlim(0.5, 1.0)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 8.8 Cefixime SHAP Explainability (GBM Model)

For the Cefixime resistance prediction task, the **Gradient Boosting Machine (GBM)** model achieved the highest test mean balanced accuracy (0.9508), outperforming other algorithms.  
Since GBM is a tree-based model, we applied **SHAP’s TreeExplainer** for efficient and accurate feature attribution.

In [ ]:
# 1) GBM model anahtarını seç (öncelik: GS_GBM_* > GBM_*)
gbm_keys = [k for k in getattr(eval_cfx, "store_models", {}).keys() if k.startswith("GS_GBM_")] \
           or [k for k in getattr(eval_cfx, "store_models", {}).keys() if k.startswith("GBM_")]
if not gbm_keys:
    raise RuntimeError("Cefixime için GBM modeli bulunamadı. Lütfen GBM eğitim hücrelerini çalıştırın.")

gbm_key   = sorted(gbm_keys)[0]
gbm_model = eval_cfx.store_models[gbm_key]

# 2) Özellik matrisi (hedef sütunu çıkar)
X_cfx = eval_cfx.X.drop(columns=[eval_cfx.target_name], errors="ignore")

# 3) SHAP TreeExplainer ve değerler
explainer = shap.TreeExplainer(gbm_model)
shap_values = explainer.shap_values(X_cfx)

# 4) Global açıklama – Beeswarm
plt.figure(figsize=(10, 6))
try:
    shap.summary_plot(shap_values[1], X_cfx, show=False)  # ikili sınıfta pozitif sınıf
except Exception:
    shap.summary_plot(shap_values, X_cfx, show=False)
plt.title("SHAP Summary – Cefixime / GBM", fontsize=14)
plt.show()

print("----------------------")
print("----------------------")
print("----------------------")

# 5) Global açıklama – Bar (mean |SHAP|)
plt.figure(figsize=(10, 6))
try:
    shap.summary_plot(shap_values[1], X_cfx, plot_type="bar", show=False)
except Exception:
    shap.summary_plot(shap_values, X_cfx, plot_type="bar", show=False)
plt.title("Mean |SHAP| – Cefixime / GBM", fontsize=14)
plt.show()

In [ ]:
sv   = shap_values[1] if isinstance(shap_values, list) else shap_values
imp  = np.abs(sv).mean(axis=0)
top10_cfx = pd.DataFrame({"feature": X_cfx.columns, "mean_abs_shap": imp}) \
                  .sort_values("mean_abs_shap", ascending=False) \
                  .head(10).reset_index(drop=True)
display(top10_cfx)

## 8.9 Cefixime Resistance – LIME Explainability (CAT MOdel)

In [ ]:
gbm_keys = [k for k in getattr(eval_cfx, "store_models", {}).keys() if k.startswith("GS_GBM_")] \
           or [k for k in getattr(eval_cfx, "store_models", {}).keys() if k.startswith("GBM_")]
if not gbm_keys:
    raise RuntimeError("No GBM model found for Cefixime.")
gbm_key   = sorted(gbm_keys)[0]
gbm_model = eval_cfx.store_models[gbm_key]

X_cfx = eval_cfx.X.drop(columns=[eval_cfx.target_name], errors="ignore").copy()
y_cfx = getattr(eval_cfx, "y", None)
class_names = ["0","1"] if y_cfx is None else [str(c) for c in sorted(pd.unique(y_cfx))]

explainer = LimeTabularExplainer(
    training_data=X_cfx.values,
    feature_names=X_cfx.columns.tolist(),
    class_names=class_names,
    mode="classification",
    discretize_continuous=True
)

instance_idx = 0
x0 = X_cfx.iloc[instance_idx].values
predict_fn = lambda data: gbm_model.predict_proba(pd.DataFrame(data, columns=X_cfx.columns))

exp = explainer.explain_instance(x0, predict_fn, num_features=10, num_samples=5000)
fig = exp.as_pyplot_figure(); _ = fig.suptitle(f"LIME – Cefixime / GBM – {gbm_key} – idx={instance_idx}", y=1.02); plt.show()

pairs = exp.as_list()
print("\nTop 10 features (LIME weights):")
for f, w in pairs: print(f"{f}  ==>  {w:+.4f}")
display(pd.DataFrame(pairs, columns=["feature_rule","lime_weight"]))